In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark/python/pyspark'

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("yarn")
         .appName("python-testing")
         .config("spark.executor.instances", 16)
         .config("spark.executor.memory", "1536m")
         #.config("spark.dynamicAllocation.enabled", "true")
         #.config("spark.executor.cores", 1)
         #.config("spark.dynamicAllocation.minExecutors", 4)
         #.config("spark.dynamicAllocation.maxExecutors", 32)
         #.config("spark.shuffle.service.enabled", "true")
         #.config("spark.shuffle.service.port", 7337)
         .getOrCreate())
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-31 13:17:10,867 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-31 13:17:17,276 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
sc.environment['PYTHONPATH'] = '/home/ubuntu/.local/lib/python3.8/site-packages'
sc.addPyFile('hbase_datasketch.py')

In [4]:
windows = sc.sequenceFile("hdfs:///files/windowed")
windows.take(1)

[(30126923,
  'CGCATCGTGGCTGGACCTGAGTCCATCTGCCCTGGTGCCTGCATGACTGGCCCTTCTCCTTCACAGACCATGGCCCCAGGCTCCCTTGCTTTCATTTCCCAGCCCGTTATTGGGGCAGGAGAGTAGCAAGCGGGGGAGTTTTGATGAGGCGAGGA')]

In [5]:
sample = windows.sample(fraction=0.0005, withReplacement=False, seed=1).cache()
sample.count()

29288

In [6]:
import datasketch as ds

In [9]:
def hashValue(value):
    import datasketch as ds
    h = ds.MinHash()
    for v in value:
        h.update(v.encode('utf8'))
    return (h.seed, h.hashvalues)

hashes = sample.mapValues(hashValue)

In [7]:
lsh = ds.MinHashLSH(threshold=0.5, num_perm=128, storage_config={
        'type': 'redis',
        'redis': {'host': 'datanode1', 'port': 6379},
        'basename': b'spark_01'
    }
)

In [12]:
def uploadFunction(variables):
    lsh = ds.MinHashLSH(threshold=0.5, num_perm=128, storage_config={
            'type': 'redis',
            'redis': {'host': '10.10.1.220', 'port': 6379},
            'basename': b'spark_01'
        }
    )
    
    h = map(lambda x: (x[0], ds.LeanMinHash(seed=x[1][0], hashvalues=x[1][1])), variables)
    for i, v in h:
        with lsh.insertion_session() as sess:
            sess.insert(i, v)

hashes.foreachPartition(uploadFunction)

2022-03-31 13:24:05,194 WARN scheduler.TaskSetManager: Lost task 2.0 in stage 4.0 (TID 212) (datanode3 executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1648460271792_0015/container_1648460271792_0015_01_000006/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1648460271792_0015/container_1648460271792_0015_01_000006/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  [Previous line repeated 1 more time]
  File "/usr/local/spark/python/pyspark/rdd.py", line 417, in func
  File "/usr/local/spark/python/pys

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 4 times, most recent failure: Lost task 3.3 in stage 4.0 (TID 250) (datanode3 executor 9): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1648460271792_0015/container_1648460271792_0015_01_000010/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/tmp/hadoop-ubuntu/nm-local-dir/usercache/ubuntu/appcache/application_1648460271792_0015/container_1648460271792_0015_01_000010/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 2918, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd

2022-03-31 13:24:07,270 WARN scheduler.TaskSetManager: Lost task 28.2 in stage 4.0 (TID 264) (datanode2 executor 3): TaskKilled (Stage cancelled)
2022-03-31 13:24:07,302 WARN scheduler.TaskSetManager: Lost task 27.1 in stage 4.0 (TID 267) (datanode2 executor 7): TaskKilled (Stage cancelled)
2022-03-31 13:24:07,312 WARN scheduler.TaskSetManager: Lost task 10.3 in stage 4.0 (TID 266) (datanode2 executor 11): TaskKilled (Stage cancelled)
2022-03-31 13:24:07,348 WARN scheduler.TaskSetManager: Lost task 0.2 in stage 4.0 (TID 268) (datanode1 executor 13): TaskKilled (Stage cancelled)
